<a href="https://colab.research.google.com/github/Foxy1987/neuroGLM/blob/master/K_fold_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning: Tuning 1D convolutional neural networks using Grid search k-fold cross-validation
__Content creator:__ David Fox

In this example, we use a grid search to evaluate different configurations for our neural network model and report on the combination that provides the best-estimated performance

The cnn.create_model_load_model() function is defined to take multiple arguments that can be optimized, all of which must have default values. This will allow us to evaluate the effect of using different optimization algorithms and weight initialization schemes for our network.

The model with default arguments is shown below:

```
def load_model(input_shape=[850, 1], trained=False, weight_path='', neurons=16, weight_constraint=1, dropout_rate=0.0,
			   kernel_size=849):
	inputs = Input(shape=input_shape)
	x = Conv1D(neurons, kernel_size=kernel_size, activation='relu',
			   name='conv1')(inputs)
	x = MaxPooling1D(name='pool')(x)
	x = Dropout(dropout_rate)(x)
	x = Flatten(name='flatten')(x)
	x = Dense(256, activation='relu', name='fc1')(x)
	x = Dense(1, name='fc2')(x)
	predictions = Activation('linear')(x)
	# predictions = Activation('sigmoid')(x)
	model = Model(outputs=predictions, inputs=inputs)

	#opt = SGD(lr=0.1, momentum=0.9)
	#model.compile(loss='mean_squared_error', optimizer=opt, metrics=[r_square, 'mse'])

	if trained:
		model.load_weights(weight_path)

	model.compile(optimizer='adam', loss='mse', metrics=[r_square, 'mse'])
	return model
```




In [2]:
#@title Clone the github repository 
!git clone https://github.com/Foxy1987/neuroGLM
%cd neuroGLM


Cloning into 'neuroGLM'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 315 (delta 171), reused 176 (delta 77), pack-reused 0
Receiving objects: 100% (315/315), 4.77 MiB | 10.46 MiB/s, done.
Resolving deltas: 100% (171/171), done.
/content/neuroGLM


In [3]:
#@title add package to google colab
import sys, os
sys.path.append(os.getcwd())

In [4]:
#@title import modules
import utils.read as io
import numpy as np
import os
from cnn.preprocessing import preprocess
from cnn.create_model import load_model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit


# Load the data

In [7]:
input_shape = [850, 1]
behaviors = ["angvturns", "vmoves", "vymoves"]
behavior_par = behaviors[2]

	# load the data from MATLAB .mat file
stim, response = io.load_behavior('datasets/behavior/control_behavior.mat', 30., 55., behavior_par, 50)
response = response.mean(axis=1)  # work on the fly-average
stim = stim[:, 0]

# preprocess for the CNN to work. This is a VERY important step!
stim_train, stim_test, resp_train, resp_test = preprocess(stim, response, input_shape)



# searching for optimal network parameters
 KerasRegressor is a wrapper for using the Scikit-Learn API with Keras models.
We pass the function our model along with parameters we wish to optimize. These are automatically bundled up and passed on to the fit() function which is called internally by the KerasRegressor class.

After creating our model, we define arrays of values for the parameter we wish to search

In this example, we use the scikit-learn TimeSeriesSplit to perform 5-fold time series cross-validation. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html



In [8]:
	model = KerasRegressor(build_fn=load_model, epochs=100, batch_size=64, verbose=1)

	# define the grid search parameters
	weight_constraint = [1, 2, 3, 4, 5]
	dropout_rate = [0, .1, .2, .3, .4, .5]
	neurons = [16, 32, 64]
	kernel_size = [749, 849]


# Grid search CV
The options are specified into a dictionary and passed to the configuration of the GridSearchCV scikit-learn class. This class will evaluate a version of our neural network model for each combination of parameters. Each combination is then evaluated using 5-fold TimeSeriesSplit cross validation.

In [9]:
param_grid = dict(dropout_rate=dropout_rate, neurons=neurons)
#param_grid = dict(kernel_size=kernel_size)
# for a single time series we want to test our model on time points in the future
# https: // scikit - learn.org / stable / modules / cross_validation.html
tscv = TimeSeriesSplit(n_splits=5)

# performing the parameter search


In [10]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=tscv, scoring='neg_mean_squared_error')

grid_result = grid.fit(stim_train, resp_train)

Epoch 1/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0911 - r_square: -0.6809 - mse: 0.0911
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0172 - r_square: 0.6793 - mse: 0.0172
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0087 - r_square: 0.8389 - mse: 0.0087
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0065 - r_square: 0.8765 - mse: 0.0065
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0057 - r_square: 0.8928 - mse: 0.0057
Epoch 6/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0047 - r_square: 0.9106 - mse: 0.0047
Epoch 7/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0037 - r_square: 0.9299 - mse: 0.0037
Epoch 8/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0033 - r_square: 0.9373 - mse: 0.0033
Epoch 9/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0032 - r_square: 0.9

# Display the best parameters
Finally, the performance and combination of configurations for the best model are displayed, followed by the performance of all combinations of parameters.

In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.031408 using {'dropout_rate': 0.2, 'neurons': 16}
-0.049908 (0.036983) with: {'dropout_rate': 0, 'neurons': 16}
-0.037677 (0.010151) with: {'dropout_rate': 0, 'neurons': 32}
-0.085844 (0.063149) with: {'dropout_rate': 0, 'neurons': 64}
-0.051102 (0.026666) with: {'dropout_rate': 0.1, 'neurons': 16}
-0.051765 (0.033009) with: {'dropout_rate': 0.1, 'neurons': 32}
-0.126717 (0.190100) with: {'dropout_rate': 0.1, 'neurons': 64}
-0.031408 (0.013183) with: {'dropout_rate': 0.2, 'neurons': 16}
-0.139452 (0.212299) with: {'dropout_rate': 0.2, 'neurons': 32}
-0.203104 (0.253109) with: {'dropout_rate': 0.2, 'neurons': 64}
-0.086999 (0.076656) with: {'dropout_rate': 0.3, 'neurons': 16}
-0.149454 (0.199839) with: {'dropout_rate': 0.3, 'neurons': 32}
-0.114761 (0.094808) with: {'dropout_rate': 0.3, 'neurons': 64}
-0.068492 (0.058758) with: {'dropout_rate': 0.4, 'neurons': 16}
-0.145936 (0.206755) with: {'dropout_rate': 0.4, 'neurons': 32}
-0.325433 (0.357313) with: {'dropout_rate': 0.4, 'n